# Retail Data Model - Simplified & Enhanced

## Overview
This notebook contains the simplified retail data model, reduced from 64 tables to 12 tables while maintaining all essential retail functionality.

## Key Achievements
- **81% complexity reduction**: From 64 to 12 tables
- **Industry-aligned naming**: Standard retail terminology
- **Fabric Lakehouse optimized**: Delta tables without unnecessary constraints
- **Complete audit trail**: CreatedBy/UpdatedBy fields on key entities
- **Soft delete capability**: IsActive flags where appropriate

## Schema Structure
- **Customer Management**: 7 tables (Customer, Location, Accounts, Relationships)
- **Product Catalog**: 2 tables (Product, Category)
- **Order Processing**: 3 tables (Order, OrderLine, OrderPayment)

## Business Value
- Faster development with simplified schema
- Lower maintenance costs
- Better performance on modern data platforms
- Enterprise-ready for retail operations

---

In [ ]:
# Simplified and enhanced data model based on out of the box Fabric Retail Data Models 

################################################################################################
# Create Schema 
################################################################################################

# Configuration
SCHEMA_NAME = "retail_simple"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {SCHEMA_NAME}")
print(f"✅ {SCHEMA_NAME} schema ready!")


################################################################################################
# Customers, Contact Info, and Locations. 7 Tables
################################################################################################

# 1. Create Customer table 
#    It can be individuals, business, or government entities
#    If it is business, then CustomerTradeName table will be used to store the trade name
#    and the first name and last name will be the main contact person for the business
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Customer (
    CustomerId STRING,
    CustomerTypeId STRING,
    IsActive BOOLEAN,
    CustomerNamePrefix STRING,
    FirstName STRING,
    LastName STRING,
    MiddleName STRING,
    Gender STRING,
    DateOfBirth DATE,
    PrimaryPhone STRING,
    SecondaryPhone STRING,
    PrimaryEmail STRING,
    SecondaryEmail STRING,
    CustomerEstablishedDate DATE,
    CustomerRelationshipTypeId STRING,
    CustomerNote STRING,
    CreatedBy STRING,
    UpdatedBy STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Customer table created!")

# 2. Create CustomerTradeName table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.CustomerTradeName (
    CustomerId STRING,
    TradeNameId STRING,
    PeriodStartDate DATE,
    PeriodEndDate DATE,
    CustomerTradeNameNote STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.CustomerTradeName table created!")

# 3. Create CustomerRelationshipType table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.CustomerRelationshipType (
    CustomerRelationshipTypeId STRING,
    CustomerRelationshipTypeName STRING,
    CustomerRelationshipTypeDescription STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.CustomerRelationshipType table created!")

# 4. Create Location table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Location (
    LocationId STRING,
    LocationName STRING,
    IsActive BOOLEAN,
    LocationAddressLine1 STRING,
    LocationAddressLine2 STRING,
    LocationCity STRING,
    LocationStateId STRING,
    LocationZipCode STRING,
    CountryId STRING,
    SubdivisionId STRING,
    LocationLatitude DECIMAL(10,7),
    LocationLongitude DECIMAL(10,7),
    LocationNote STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Location table created!")

# 5. Create CustomerLocation table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.CustomerLocation (
    CustomerId STRING,
    LocationId STRING,
    PeriodStartDate DATE,
    PeriodEndDate DATE
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.CustomerLocation table created!")

# 6. Create CustomerAccount table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.CustomerAccount (
    CustomerAccountId STRING,
    ParentAccountId STRING,
    CustomerAccountName STRING,
    CustomerAccountTypeId STRING,
    CustomerAccountOpenedDate DATE,
    CustomerAccountClosedDate DATE,
    CustomerAccountApplicationNumber STRING,
    CustomerId STRING,
    ResponsibilityCenterId STRING,
    LedgerId STRING,
    LedgerAccountNumber STRING,
    IsoCurrencyCode STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.CustomerAccount table created!")

# 7. Create CustomerAccountLocation table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.CustomerAccountLocation (
    CustomerAccountId STRING,
    LocationId STRING,
    PeriodStartDate DATE,
    PeriodEndDate DATE
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.CustomerAccountLocation table created!")

################################################################################################
# Products - 2 Tables
################################################################################################

# 8. Create Product table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Product (
    ProductId STRING,
    ProductName STRING,
    ProductDescription STRING,
    BrandName STRING,
    ProductNumber STRING,
    Color STRING,
    ProductModel STRING,
    ProductCategoryID STRING,
    ListPrice DECIMAL(18,2),
    StandardCost DECIMAL(18,2),
    Weight DECIMAL(18,3),
    WeightUom STRING,     -- kg, lb, oz
    ProductStatus STRING, -- active, inactive, discontinued
    CreatedDate DATE,
    SellStartDate DATE,
    SellEndDate DATE,
    IsoCurrencyCode STRING,
    UpdatedDate DATE,
    CreatedBy STRING,
    UpdatedBy STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Product table created!")

# 9. Create Brand table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Category(
    CategoryId STRING,
    ParentCategoryId STRING,
    CategoryDescription STRING,
    BrandName STRING,
    BrandLogoUrl STRING,
    IsActive BOOLEAN
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Category table created!")


################################################################################################
# Orders - 3 Tables
################################################################################################

# 10. Create Order table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Order (
    OrderId STRING,       -- Unique identifer like UUID
    OrderNumber STRING,   -- Customer-facing order number
    CustomerId STRING,
    CustomerAccountId STRING,
    OrderDate DATE,
    OrderStatus STRING,
    OrderTotal DECIMAL(18,2),
    SubTotal DECIMAL(18,2),
    TaxAmount DECIMAL(18,2),
    ShippingAmount DECIMAL(18,2), -- ShipingCost, Standard retail transaction terminology
    DiscountAmount DECIMAL(18,2),
    PaymentMethod STRING,
    ShippingAddress STRING,
    BillingAddress STRING,
    OrderNotes STRING,
    CreatedTimestamp TIMESTAMP,
    UpdatedTimestamp TIMESTAMP,
    IsoCurrencyCode STRING,
    StoreId STRING,
    Status STRING,
    SalesChannelId STRING,
    CreatedBy STRING 
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Order table created!")

# 11. Create OrderLine table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.OrderLine (
    OrderId STRING,
    OrderLineNumber INT,
    ProductId STRING,
    ProductName STRING,
    Quantity DECIMAL(18,2),
    UnitPrice DECIMAL(18,2),
    LineTotal DECIMAL(18,2),
    DiscountAmount DECIMAL(18,2),
    TaxAmount DECIMAL(18,2)
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.OrderLine table created!")

# 12. Create OrderPayment table
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.OrderPayment (
    OrderId STRING,
    PaymentId STRING,
    PaymentMethod STRING,
    PaymentAmount DECIMAL(18,2),
    PaymentDate DATE,
    PaymentStatus STRING,
    TransactionId STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.OrderPayment table created!")

print(f"✅ All {SCHEMA_NAME} tables created successfully!")
print(f"📊 Total tables created: 12")